In [48]:
'''
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.
'''

'\nEscreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e criar um pandas df em que cada registro tem a coluna paper_id, que é um número sequencial.\n'

In [49]:
tei_path = "../data/interim/tei"

In [50]:
import re


def extract_year(citation):
    if citation is None:
        return (None, None)

    match = re.search(r'\b(\d{4})\b', citation)
    return (int(match.group(1)), None) if match else (None, None)


def extract_year_and_page(citation):

    if citation is None:
        return (None, None)

    # 🔴 CASO ESPECIAL: [ano original] ano edição, p./pp./:
    # Ex:
    # (Mises [1949] 1963: 251-255)
    # Mises ([1936] 1951, p. 287)
    special = re.search(
        r"\[\s*(\d{4})\s*\]\s*\d{4}[a-z]?\s*(?:[:;,]|\s)\s*"
        r"(?:pp?\.\s*)?(\d+)",
        citation
    )
    if special:
        return int(special.group(1)), int(special.group(2))

    # 🔧 Normaliza [1949], [[1949]], etc. (para os demais casos)
    citation = re.sub(r"\[\s*(\d{4})\s*\]+", r"\1", citation)

    match = re.search(
        # Ano + pp./p.
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?)[:\s,]*pp?\.\s*(\d+)(?:-\d+)?|"

        # Ano: página
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?):\s*(\d+)|"

        # Ano, p. página
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?),\s*p\.\s*(\d+)|"

        # Ano, intervalo
        r"(?:\b|[^\(])(?:[^,]*, )?(\d{4}[a-z]?),\s*(\d+)-\d+|"

        # Ano, página
        r"(?:\b|[^\(])(?:[^,]* )?(\d{4}[a-z]?),?\s*(\d+)(?:-\d+)?|"

        # Somente pp. página
        r"(?:\b|[^\(])(?:[^,]*)pp?\.\s*(\d+)",
        citation
    )

    if match:
        groups = match.groups()

        # 🔹 Pares (year, page)
        for i in range(0, len(groups) - 1, 2):
            if groups[i] and groups[i + 1]:
                year = int(re.match(r"\d{4}", groups[i]).group())
                page = int(groups[i + 1])
                return year, page

        # 🔹 Apenas página
        for g in groups:
            if g and g.isdigit():
                return None, int(g)

    return extract_year(citation)



   

# Test the function
assert extract_year_and_page("Mises (1949, p.258)") == (1949, 258)
assert extract_year_and_page("(Mises, 1996, pp. 538-86)") == (1996, 538)
assert extract_year_and_page("(von Mises, 1963, p.254)") == (1963, 254)
assert extract_year_and_page("(Mises, 1920, 121-122)") == (1920, 121)
assert extract_year_and_page("(Mises 1949, 236-237)") == (1949, 236)
assert extract_year_and_page("(Mises 1920, 109)") == (1920, 109)
assert extract_year_and_page("(Mises 1920, p.162)") == (1920, 162)
assert extract_year_and_page("(von Mises, 1949: 351)") == (1949, 351)
assert extract_year_and_page("(Mises 1966: 493)") == (1966, 493)
assert extract_year_and_page("(von Mises 1998, p. 270)") == (1998, 270)
assert extract_year_and_page("Mises 1949, p. 3)") == (1949, 3)
assert extract_year_and_page("Mises 1985b, p. 236") == (1985, 236)
assert extract_year_and_page("Mises 1957b, 372") == (1957, 372)
assert extract_year_and_page("Mises, pp. 105-6;") == (None, 105)
assert extract_year_and_page("(Mises, 1949, p. 3)") == (1949, 3)
assert extract_year_and_page("(L Von Mises 1949, pp. 393)") == (1949, 393)
assert extract_year_and_page("(L Von Mises 1949 , pp. 393)") == (1949, 393)
assert extract_year_and_page("(C Berg 2022)") == (2022, None)
assert extract_year_and_page("Hayek (1976:71)") == (1976, 71)
assert extract_year_and_page("According to Mises ([1949] 1998: 116)") == (1949, 116)
assert extract_year_and_page(" Mises ( [1949]], p. 405)") == (1949, 405)
assert extract_year_and_page("(Mises [1949] 1963: 251-255)")  == (1949, 251)
assert extract_year_and_page("Mises ([1936] 1951, p. 287)") == (1936, 287)

assert extract_year_and_page(None) == (None, None)
assert extract_year_and_page("[and]") == (None, None)


In [51]:
import re

def extract_author(ref):
    if ref is None:
        return None

    match = re.search(
        r"([A-Za-z][A-Za-z\s\.']+?)"
        r"(?:\s+(?:and|And|&)\s+[A-Za-z][A-Za-z\s\.']+)?"
        r"(?=\s\d{4}|\s\[|\s\(|,|:|\))",
        ref
    )

    if match:
        name = match.group(1).strip()

        # Remove possessive 's
        name = re.sub(r"'s$", "", name)

        lowercase_particles = {"de", "der", "von"}
        ignore_words = {"et", "al."}

        def normalize(word):
            w = word.lower()
            if w in lowercase_particles:
                return w
            if w in ignore_words:
                return word
            if w.startswith("mc") and len(w) > 2:
                return "Mc" + w[2].upper() + w[3:]
            return word.capitalize()

        temp = " ".join(normalize(word) for word in name.split())

        temp = (
            temp.replace("von Mises", "Mises")
                .replace("Von Mises", "Mises")
                .replace("L V Mises", "Mises")
                .replace("L Mises", "Mises")
                .replace("L Von Mises", "Mises")
                .replace("von Hayek", "Hayek")
                .replace("Von Hayek", "Hayek")
                .replace("F A Hayek", "Hayek")
                .replace("F. A. Hayek", "Hayek")
                .replace("K Marx", "Marx")
                .replace(" et al.", "")
        )

        return temp.strip()

    match_single_word = re.search(
        r"([A-Za-z]+(?:'s)?)(?=\s\d{4}|\s\(|:|\))",
        ref
    )

    if match_single_word:
        name = match_single_word.group(1).replace("'s", "")
        return name.capitalize()

    return None




# Test cases
assert extract_author("Johnson, 1999;") == "Johnson"             
assert extract_author("Jouvenel (1961)") == "Jouvenel"
assert extract_author("Keen 2011") == "Keen"
assert extract_author("(Menger, [1981])") == "Menger"
assert extract_author("von Mises (1949)") == "Mises"
assert extract_author("Von Hayek (1949)") == "Hayek"
assert extract_author("de Broglie (1924)") == "de Broglie"
assert extract_author("Allen 2005)") == "Allen"
assert extract_author("Folta's (1998)") == "Folta"
assert extract_author("Boettke et al. (1998)") == "Boettke"
assert extract_author("Floss and Klein (1998)") == "Floss"
assert extract_author("Floss And Klein (1998)") == "Floss"
assert extract_author("(Von Mises 1949 )") == "Mises"
assert extract_author("(L V Mises 1949 )") == "Mises" 
assert extract_author("(L Mises 1998 )") == "Mises"
assert extract_author("(L Von Mises 1998 )") == "Mises"
assert extract_author("Van der Waals (1873)") == "Van der Waals"
assert extract_author("(McGrath et al., 2004: 96)") == "McGrath"
assert extract_author("Shane and Venkataraman's (2000)") == "Shane"
assert extract_author("(Alvarez & Barney, 2007") == "Alvarez"


assert extract_author(None) == None



In [52]:
'''
prompt:
Gere código python que cria uma classe Reference que tem atributos: raw, context, sentence_seq_number, sentence_id, author, page, year. 
raw, context, e sentence_id são passados pelo construtor.
page e year são obtidos a partir da chamada à extract_year_and_page(raw), que retorna uma tupla (year, page).
author é obtido a partir da chamada a extract_author(raw).
'''

class Reference:
    def __init__(self, raw, context, sentence_seq_number, reference_seq_number, sentence_id, paragraph_id, head_id):
        self.raw = self.expand_raw(raw, context)
        self.context = context
        self.sentence_seq_number = sentence_seq_number
        self.reference_seq_number = reference_seq_number
        self.sentence_id = sentence_id
        self.paragraph_id = paragraph_id
        self.head_id = head_id
        self.co_cited_count = None

        self.year, self.page = extract_year_and_page(self.raw)
        self.author = extract_author(self.raw)

    def expand_raw(self, raw, context):
        """
        Se raw contém '(' mas não ')', tenta expandi-lo usando o contexto
        até o primeiro ')' encontrado após a ocorrência de raw.
        """
        if raw is None or context is None:
            return raw

        if "(" not in raw or ")" in raw:
            return raw

        idx = context.find(raw)
        if idx == -1:
            return raw

        after = context[idx:]
        close_paren = after.find(")")
        
        if close_paren == -1:
            return raw

        return after[: close_paren + 1]


    def __repr__(self):
        return (f"Reference(raw={self.raw!r}, context={self.context!r} author={self.author!r}"
                f"sentence_id={self.sentence_id!r}, year={self.year!r}, page={self.page!r}), co-cited={self.co_cited_count!r}")



ref = Reference("(L Von Mises 1949 , pp. 393)", "",  3, 1, "123", "124", "125")
assert ref.year == 1949
assert ref.page == 393
assert ref.sentence_seq_number == 3
assert ref.reference_seq_number == 1
assert ref.author == "Mises"
assert ref.sentence_id == "123"
assert ref.paragraph_id == "124"
assert ref.head_id == "125"
assert ref.co_cited_count == None


ref = Reference('Mises ( [1949]]', 'In a striking section of his Human Action, von Mises (1966Mises ( [1949]], p. 405) discusses the "epistemological import" of Menger\'s theory of money.', 1, 1, '123', '124', '125')
assert ref.author == "Mises"
assert ref.year == 1949
assert ref.page == 405
assert ref.co_cited_count == None

ref = Reference("(Ramoglou & McMullen, 2021)", "As importantly, the creation of a constituent condition of the possibility of entrepreneurial success cannot be the creation of opportunity, since \"an opportunity\" is never a singular condition (Ramoglou & McMullen, 2021) but a collection of \"opportunity ingredients\" (Ramoglou, 2021a).", 3, 1, "123", "124", "125")
assert ref.year == 2021
assert ref.co_cited_count == None

ref = Reference(None, None, None, None, None, None, None)
assert ref.year == None
assert ref.page == None
assert ref.sentence_seq_number == None
assert ref.author == None
assert ref.co_cited_count == None
assert print(ref) == None


Reference(raw=None, context=None author=Nonesentence_id=None, year=None, page=None), co-cited=None


In [53]:
def deduplica_refs(references):
    """
    Retorna uma lista com no máximo uma Reference por autor.

    Regra de desempate (do mais importante ao menos):
    1. Prefere referência com page
    2. Se ambas têm page e ano conhecido, prefere ano mais antigo
    3. Se mesmo ano (ou ambos sem ano), prefere menor página
    4. Se nenhuma tem page:
       - se ambos têm ano, prefere o mais antigo
       - caso contrário, mantém a atual
    """
    best_by_author = {}

    for ref in references:
        author = ref.author

        if author not in best_by_author:
            best_by_author[author] = ref
            continue

        current = best_by_author[author]

        # 1. current sem page, ref com page
        if current.page is None and ref.page is not None:
            best_by_author[author] = ref
            continue

        # 2. ambos têm page
        if current.page is not None and ref.page is not None:
            # 2a. ambos têm ano → ano mais antigo vence
            if current.year is not None and ref.year is not None:
                if ref.year < current.year:
                    best_by_author[author] = ref
                    continue
                if ref.year == current.year and ref.page < current.page:
                    best_by_author[author] = ref
                    continue

            # 2b. algum (ou ambos) sem ano → menor página vence
            if ref.page < current.page:
                best_by_author[author] = ref

            continue

        # 3. nenhum tem page
        if current.page is None and ref.page is None:
            # só compara ano se ambos existirem
            if current.year is not None and ref.year is not None:
                if ref.year < current.year:
                    best_by_author[author] = ref

    return list(best_by_author.values())



def test_single_reference():
    ref = Reference("(L Von Mises 1949)", "", 3, 1, "123", "123", "123")
    ref.author = "Mises"
    ref.page = 10

    result = deduplica_refs([ref])

    assert len(result) == 1
    assert result[0].author == "Mises"
    assert result[0].page == 10


def test_same_author_prefers_with_page():
    ref1 = Reference("(L Von Mises 1949)", "", 1, 1, "1", "1", "1")
    ref1.author = "Mises"
    ref1.page = None

    ref2 = Reference("(L Von Mises 1949, p. 393)", "", 2, 1, "2", "1", "1")
    ref2.author = "Mises"
    ref2.page = 393

    result = deduplica_refs([ref1, ref2])

    assert len(result) == 1
    assert result[0].author == "Mises"
    assert result[0].page == 393


def test_same_author_all_without_page():
    ref1 = Reference("(L Von Mises 1949)", "", 1, 1, "1", "1", "1")
    ref1.author = "Mises"
    ref1.page = None

    ref2 = Reference("(L Von Mises 1950)", "", 2, 1, "2", "2", "12")
    ref2.author = "Mises"
    ref2.page = None

    result = deduplica_refs([ref1, ref2])

    assert len(result) == 1
    assert result[0].author == "Mises"
    assert result[0].page is None


def test_multiple_authors():
    ref1 = Reference("(L Von Mises 1949)", "", 1, 1, "1", "1", "1")
    ref1.author = "Mises"
    ref1.page = None

    ref2 = Reference("(Hayek 1945, p. 12)", "", 2, 1, "2", "2", "2")
    ref2.author = "Hayek"
    ref2.page = 12

    ref3 = Reference("(L Von Mises 1949, p. 393)", "", 3, 1, "3", "2", "1")
    ref3.author = "Mises"
    ref3.page = 393

    result = deduplica_refs([ref1, ref2, ref3])

    assert len(result) == 2

    by_author = {r.author: r for r in result}

    assert by_author["Mises"].page == 393
    assert by_author["Hayek"].page == 12


def test_first_page_wins():
    ref1 = Reference("(Mises 1949, p. 10)", "", 1, 1, "1", "1", "1")
    ref1.author = "Mises"
    ref1.page = 10

    ref2 = Reference("(Mises 1949, p. 20)", "", 2, 1, "2", "2", "2")
    ref2.author = "Mises"
    ref2.page = 20

    result = deduplica_refs([ref1, ref2])

    assert len(result) == 1
    assert result[0].page == 10


def test_older_book_wins():
    ref1 = Reference("(Mises 1966, p. 10)", "", 1, 1, "1", "1", "1")
    ref1.author = "Mises"
    ref1.page = 10

    ref2 = Reference("(Mises 1949, p. 20)", "", 2, 1, "2", "2", "2")
    ref2.author = "Mises"
    ref2.page = 20

    result = deduplica_refs([ref1, ref2])

    assert len(result) == 1
    assert result[0].year == 1949
    assert result[0].page == 20


def test_no_years():
    ref1 = Reference("(Mises, p. 10)", "", 1, 1, "1", "1", "1")
    ref1.author = "Mises"
    ref1.page = 10

    ref2 = Reference("(Mises, p. 20)", "", 2, 1, "2", "2", "1")
    ref2.author = "Mises"
    ref2.page = 20

    result = deduplica_refs([ref1, ref2])

    assert len(result) == 1
    assert result[0].year == None
    assert result[0].page == 10

# =========================
# Runner simples
# =========================

test_single_reference()
test_same_author_prefers_with_page()
test_same_author_all_without_page()
test_multiple_authors()
test_first_page_wins()
test_older_book_wins()
test_no_years()

print("All tests passed!")


All tests passed!


In [54]:
from lxml import etree
from typing import List, Tuple


def parse_tei(tei_filepath: str) -> Tuple[str, int, int, List[Reference]]:
    parser = etree.XMLParser(ns_clean=True)
    tree = etree.parse(tei_filepath, parser)
    root = tree.getroot()

    # Namespace TEI
    nsmap = root.nsmap.copy()
    nsmap['tei'] = nsmap.get(None, 'http://www.tei-c.org/ns/1.0')

    # 1. Paper title
    title_xpath = './/tei:teiHeader/tei:fileDesc/tei:titleStmt/tei:title'
    title_elem = root.find(title_xpath, namespaces=nsmap)
    title = (
        title_elem.text.strip()
        if title_elem is not None and title_elem.text is not None
        else "Unknown Title"
    )

    # 2. Count <s>
    s_xpath = './/tei:s'
    s_elems = root.findall(s_xpath, namespaces=nsmap)
    num_s = len(s_elems)

    # 3. Count <biblStruct>
    bibl_xpath = './/tei:biblStruct'
    bibl_elems = root.findall(bibl_xpath, namespaces=nsmap)
    num_bibl = len(bibl_elems)

    # 4. References
    references = []
    ref_idx = 0

    for sentence_seq, s in enumerate(s_elems, start=1):

        # sentence_id
        sentence_id = s.get('{http://www.w3.org/XML/1998/namespace}id')

        # paragraph_id
        paragraph_id = None
        p = s.getparent()
        if p is not None and p.tag.endswith('p'):
            paragraph_id = p.get('{http://www.w3.org/XML/1998/namespace}id')

        # section logic (TEI-correct)
        # 1) sobe até o <div> mais próximo
        # 2) procura <head> como filho do <div>
        # 3) fallback: usa o <div>
        head_id = None

        div_ancestor = None
        for ancestor in s.iterancestors():
            if ancestor.tag.endswith('div'):
                div_ancestor = ancestor
                break

        if div_ancestor is not None:
            # procura <head> como filho direto do <div>
            head_elem = div_ancestor.find('tei:head', namespaces=nsmap)
            if head_elem is not None:
                head_id = head_elem.get('{http://www.w3.org/XML/1998/namespace}id')
            else:
                # fallback: usa o próprio <div>
                head_id = div_ancestor.get('{http://www.w3.org/XML/1998/namespace}id')

        # contexto = texto da sentença
        context_text = ''.join(s.itertext()).strip()

        local_references = []
        for ref in s.findall('.//tei:ref', namespaces=nsmap):
            ref_idx += 1
            ref_text = ''.join(ref.itertext()).strip()

            local_references.append(
                Reference(
                    raw=ref_text,
                    sentence_seq_number=sentence_seq,
                    reference_seq_number=ref_idx,
                    sentence_id=sentence_id,
                    paragraph_id=paragraph_id,
                    head_id=head_id,
                    context=context_text,
                )
            )

        local_references = deduplica_refs(local_references)

        for ref in local_references:
            ref.co_cited_count = len(local_references) - 1

        references.extend(local_references)

    return title, num_s, num_bibl, references


In [55]:
# TESTE 1

paper_path = "../data/interim/tei/A-Historical-Intervention-in-the-Opportunity-Wars-Forgotten-Scholarship-the-DiscoveryCreation-Disruption-and-Moving-Forward-by-Looking-Backward_2023_SAGE-Publications-Ltd.pdf.grobid.tei.xml"
title, sentence_count, reference_count, refs = parse_tei(paper_path) 

assert sentence_count == 283
assert reference_count == 106
assert len(refs) == 204
assert title == "A Historical Intervention in the \"Opportunity Wars\": Forgotten Scholarship, the Discovery/Creation Disruption, and Moving Forward by Looking Backward"

assert refs[0].sentence_seq_number == 7
assert refs[0].reference_seq_number == 1
assert refs[0].sentence_id == '_paHYmXc'
assert refs[0].paragraph_id == "_byrr9Qs"
assert refs[0].head_id == "_Vbdbk8s"
assert refs[0].co_cited_count == 0

target = "Besides Cole's (1959) early discussion"
matches = [ref for ref in refs if ref.context.startswith(target)]
assert len(matches) == 6
assert matches[0].sentence_seq_number == 36


target = "Furthermore, one should not neglect the wealth"
matches = [ref for ref in refs if ref.context.startswith(target)]
assert len(matches) == 2
assert matches[0].sentence_seq_number == 37



In [56]:
# TESTE 2

paper_path = "../data/interim/tei/10.1108.17506221211282000.pdf.grobid.tei.xml"
title, sentence_count, reference_count, refs = parse_tei(paper_path) 

refs = [ref for ref in refs if ref.sentence_id == "_sYcZ3cm"]

assert len(refs) == 1

assert refs[0].author == "Mises"
assert refs[0].page == 417
assert refs[0].co_cited_count == 0

assert refs[0].head_id == "_GaeQjWv"



In [57]:
# TESTE 3

paper_path = "../data/interim/tei/10.1002.9780470999059.ch17.pdf.grobid.tei.xml"
title, sentence_count, reference_count, refs = parse_tei(paper_path) 

refs = [ref for ref in refs if ref.sentence_id == "_szF6vj8"]

refs

[Reference(raw='Mises (1966Mises ( [1949]], p. 405)', context='In a striking section of his Human Action, von Mises (1966Mises ( [1949]], p. 405) discusses the "epistemological import" of Menger\'s theory of money.' author='Mises'sentence_id='_szF6vj8', year=1949, page=405), co-cited=0]

In [58]:
# TESTE 4

paper_path = "../data/interim/tei/10.1007.978-3-030-05557-8.pdf.grobid.tei.xml"

title, sentence_count, reference_count, refs = parse_tei(paper_path) 

refs = [ref for ref in refs if ref.sentence_id == "_Gk3c4QK"]

assert len(refs) == 1
assert refs[0].author == "Mises"
assert refs[0].year == 1949
assert refs[0].page == 107
assert refs[0].co_cited_count == 0



In [59]:
'''
prompt:
Escreva código python que escreve uma função read_tei_papers(path) que lê o nome de todos os arquivos XML de path e cria dois dataframes.
Um pandas df chamado "papers_df" em que cada registro tem a coluna paper_id, que é um número sequencial, além das colunas title, filename, sentence_count, reference_count.
Para cada arquivo, invoque parse_tei(tei_filepath), que retorna sentence_count, reference_count e refs.  
Adicione sentence_count e reference_count no papers_df.  
Add a try-catch loop that catches exceptions in XML Parsing.
Ao outro df, chamado refs_df, adicione todos os refs. Um ref é um objeto Reference que tem os campos raw, context, sentence_id, sentence_seq_number e page.
Cada campo deve ser uma coluna em refs_df.
'''

import os
import pandas as pd
from xml.etree.ElementTree import ParseError


def read_tei_papers(path: str):
    papers = []
    refs = []

    paper_id = 1

    for filename in os.listdir(path):
        if filename.endswith(".xml"):
            tei_filepath = os.path.join(path, filename)
            try:
                title, sentence_count, reference_count, ref_list = parse_tei(tei_filepath)

                # Adiciona entrada ao papers_df
                papers.append({
                    "paper_id": paper_id,
                    "title": title,
                    "filename": filename,
                    "sentence_count": sentence_count,
                    "reference_count": reference_count
                })

                # Adiciona entradas ao refs_df
                for ref in ref_list:
                    refs.append({
                        "paper_id": paper_id,
                        "raw": ref.raw,
                        "context": ref.context,
                        "co_cited_count": ref.co_cited_count,
                        "head_id": ref.head_id,
                        "paragraph_id": ref.paragraph_id,
                        "sentence_id": ref.sentence_id,
                        "sentence_seq_number": ref.sentence_seq_number,
                        "reference_seq_number": ref.reference_seq_number,
                        "author": ref.author,
                        "page": ref.page,
                        "year": ref.year
                    })

                paper_id += 1

            except ParseError as e:
                print(f"Erro ao processar {filename}: {e}")
            except Exception as e:
                print(f"Erro desconhecido em {filename}: {e}")

    # Cria os DataFrames
    papers_df = pd.DataFrame(papers)
    refs_df = pd.DataFrame(refs)

    return papers_df, refs_df

In [60]:
import pandas as pd


scopus_df = pd.read_csv("../data/raw/scopus.csv")

scopus_df['Source title'] = scopus_df['Source title'].replace("The Review of Austrian Economics", "Review of Austrian Economics")

scopus_df.head()

,Title,Source title,DOI,Link,Author Keywords,Publisher,Open Access
0,Two worlds collide: A review essay of Humanomi...,Review of Austrian Economics,10.1007/s11138-022-00610-y,https://www.scopus.com/inward/record.uri?eid=2...,Adam Smith; B12; B13; B25; B53; Behavioral eco...,Springer,NaN
1,MODERN AUSTRIAN ECONOMICS: Archaeology of a Re...,Modern Austrian Economics: Archaeology of a Re...,10.4324/9781003549666,https://www.scopus.com/inward/record.uri?eid=2...,NaN,Taylor and Francis,NaN
2,Artificial intelligence and economic planning,AI and Society,10.1007/s00146-022-01523-x,https://www.scopus.com/inward/record.uri?eid=2...,Artificial intelligence; Economic planning; H1...,Springer Science and Business Media Deutschlan...,NaN
3,"""WHAT IS AN OPPORTUNITY?"": FROM THEORETICAL MY...",Academy of Management Review,10.5465/amr.2020.0335,https://www.scopus.com/inward/record.uri?eid=2...,NaN,Academy of Management,All Open Access; Green Open Access
4,How Organizational Inefficiency Adversely Affe...,"Studia Universitatis Vasile Goldis Arad, Econo...",10.2478/sues-2024-0017,https://www.scopus.com/inward/record.uri?eid=2...,asset size; capital market; firm size; mission...,Sciendo,All Open Access; Gold Open Access


In [61]:
papers_df, refs_df = read_tei_papers(tei_path)

assert papers_df['paper_id'].is_unique, "Duplicate paper_id values found in papers_df"

print("Read TEI files!")

papers_df.shape

Erro desconhecido em 10.1002.9781118867938.pdf.grobid.tei.xml: Start tag expected, '<' not found, line 1, column 1 (../data/interim/tei/10.1002.9781118867938.pdf.grobid.tei.xml, line 1)
Erro desconhecido em 10.1002.9781119201366.pdf.grobid.tei.xml: Start tag expected, '<' not found, line 1, column 1 (../data/interim/tei/10.1002.9781119201366.pdf.grobid.tei.xml, line 1)
Erro desconhecido em 10.1002.sej.1443.pdf.grobid.tei.xml: Start tag expected, '<' not found, line 1, column 1 (../data/interim/tei/10.1002.sej.1443.pdf.grobid.tei.xml, line 1)
Erro desconhecido em 10.1007.978-3-030-25328-8.pdf.grobid.tei.xml: Start tag expected, '<' not found, line 1, column 1 (../data/interim/tei/10.1007.978-3-030-25328-8.pdf.grobid.tei.xml, line 1)
Erro desconhecido em 10.1007.978-3-030-48667-9.pdf.grobid.tei.xml: Start tag expected, '<' not found, line 1, column 1 (../data/interim/tei/10.1007.978-3-030-48667-9.pdf.grobid.tei.xml, line 1)
Erro desconhecido em 10.1007.978-3-030-62962-5_3.pdf.grobid.tei.

(2698, 5)

In [62]:
!pip install rapidfuzz

In [63]:
import pandas as pd
from rapidfuzz import process, fuzz

# Normalização básica
papers_df["title_norm"] = papers_df["title"].str.lower().str.strip()
scopus_df["title_norm"] = scopus_df["Title"].str.lower().str.strip()

scopus_titles = scopus_df["title_norm"].tolist()

def get_best_scopus_source(title):
    if pd.isna(title):
        return pd.Series([None, None])

    match, score, idx = process.extractOne(
        title,
        scopus_titles,
        scorer=fuzz.token_sort_ratio  # melhor para títulos acadêmicos
    )

    source_title = scopus_df.iloc[idx]["Source title"]
    return pd.Series([source_title, score])

# Aplica fuzzy matching
papers_df[["source title", "similarity"]] = (
    papers_df["title_norm"]
    .apply(get_best_scopus_source)
)

# (Opcional) filtro de qualidade
SIMILARITY_THRESHOLD = 85
papers_df.loc[papers_df["similarity"] < SIMILARITY_THRESHOLD, "scopus_source_title"] = None

# Limpeza
papers_df = papers_df.drop(columns=["title_norm"])
papers_df = papers_df.drop(columns=["scopus_source_title"])

papers_df.head()

,paper_id,title,filename,sentence_count,reference_count,source title,similarity
0,1,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444
1,2,THE EARLIER HISTORY OF THE AUSTRIAN SCHOOL,10.1002.9780470999059.ch28.pdf.grobid.tei.xml,143,30,The Palgrave Handbook of the History of Human ...,71.264368
2,3,Unknown Title,10.1002.9781118324950.ch15.pdf.grobid.tei.xml,1088,68,Review of Austrian Economics,44.444444
3,4,Unknown Title,10.1002.9781118867808.pdf.grobid.tei.xml,5115,164,Review of Austrian Economics,44.444444
4,5,Financial Stability Stability,10.1002.9781118935835.pdf.grobid.tei.xml,3394,31,Institutions in Crisis: European Perspectives ...,63.333333


In [64]:
#papers_df = papers_df.drop(columns=['Title'])
papers_df = papers_df.sort_values(by='paper_id', ascending=False)

refs_df = pd.merge(refs_df, papers_df, on='paper_id', how='left')

print("Creating spreadsheets...")
papers_df.to_csv("../data/processed/papers.csv", index=False)
refs_df.to_csv("../data/processed/refs.csv", index=False)

na_percentage = papers_df['source title'].isna().mean() * 100
print(f"Percentage of NA in 'source title': {na_percentage:.2f}%")


Creating spreadsheets...
Percentage of NA in 'source title': 0.00%


In [65]:
papers_df.tail()

,paper_id,title,filename,sentence_count,reference_count,source title,similarity
4,5,Financial Stability Stability,10.1002.9781118935835.pdf.grobid.tei.xml,3394,31,Institutions in Crisis: European Perspectives ...,63.333333
3,4,Unknown Title,10.1002.9781118867808.pdf.grobid.tei.xml,5115,164,Review of Austrian Economics,44.444444
2,3,Unknown Title,10.1002.9781118324950.ch15.pdf.grobid.tei.xml,1088,68,Review of Austrian Economics,44.444444
1,2,THE EARLIER HISTORY OF THE AUSTRIAN SCHOOL,10.1002.9780470999059.ch28.pdf.grobid.tei.xml,143,30,The Palgrave Handbook of the History of Human ...,71.264368
0,1,Unknown Title,10.1002.9780470999059.ch17.pdf.grobid.tei.xml,298,20,Review of Austrian Economics,44.444444


In [66]:
paper = papers_df[papers_df.title == 'A Historical Intervention in the "Opportunity Wars": Forgotten Scholarship, the Discovery/Creation Disruption, and Moving Forward by Looking Backward'].iloc[0]

assert paper['source title'] == 'Entrepreneurship: Theory and Practice'
